In [42]:
import pandas as pd


train=pd.read_csv("electric_train.csv")


# drop first column
train.drop(train.columns[0], axis=1, inplace=True)

In [43]:
test=pd.read_csv("electric_test.csv")
# drop first column
test.drop(test.columns[0], axis=1, inplace=True)

In [44]:
train.columns
for idx, col in enumerate(train.columns):
    print(idx, col)
    train.rename(columns={col: col.replace("electric_train.", "")}, inplace=True)

0 electric_train.num
1 electric_train.tm
2 electric_train.hh24
3 electric_train.n
4 electric_train.stn
5 electric_train.sum_qctr
6 electric_train.sum_load
7 electric_train.n_mean_load
8 electric_train.nph_ta
9 electric_train.nph_hm
10 electric_train.nph_ws_10m
11 electric_train.nph_rn_60m
12 electric_train.nph_ta_chi
13 electric_train.weekday
14 electric_train.week_name
15 electric_train.elec


In [45]:
train.columns

Index(['num', 'tm', 'hh24', 'n', 'stn', 'sum_qctr', 'sum_load', 'n_mean_load',
       'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'weekday',
       'week_name', 'elec'],
      dtype='object')

In [46]:
# split datetime tm into date and time
train['hour'] = pd.to_datetime(train['tm']).dt.hour   
train['month'] = pd.to_datetime(train['tm']).dt.month
train['day'] = pd.to_datetime(train['tm']).dt.day

In [47]:
train.drop('tm', axis=1, inplace=True)

In [48]:
train

,num,hh24,n,stn,sum_qctr,sum_load,n_mean_load,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elec,hour,month,day
0,4821,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,1,1,1
1,4821,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2,1,1
2,4821,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,3,1,1
3,4821,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,4,1,1
4,4821,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,20,23,671,34200,6779.84,225.462000,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,20,12,31
7593351,20947,21,23,671,34200,6802.40,225.462000,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,21,12,31
7593352,20947,22,23,671,34200,6706.68,225.462000,2.4,47.4,2.1,0.0,0.2,5,1,129.33,22,12,31
7593353,20947,23,23,671,34200,6355.88,225.462000,2.5,47.0,2.1,0.0,0.3,5,1,122.57,23,12,31


In [49]:
train.drop(columns=['sum_qctr','sum_load','n_mean_load'], inplace=True)
# take log on nph_ta , nph_hm, nph_ws_10m, nph_rn_60m, nph_ta_chi
# take log and if values are negative take log(abs) and take netaive again,  and if values are 0, take 0
# and add new columns with _log
import numpy as np
for col in ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']:
    train[col+'_log'] = np.log(train[col].abs()+1)
    train[col+'_log'] = np.where(train[col+'_log']<0, -train[col+'_log'], train[col+'_log'])
    train[col+'_log'] = np.where(train[col+'_log']==0, 0, train[col+'_log'])



In [60]:
from autogluon.tabular import TabularDataset, TabularPredictor

#train = pd.read_csv('../preprocess/q1_train_data_2.csv')
#test = pd.read_csv('../preprocess/q1_test_data_2.csv')

train_data = TabularDataset(train)
#test_data = TabularDataset(test)

print("==================Tabular_complete========================")

hyperparameters = {'GBM':{}}

save_path = './models'
predictor = TabularPredictor(label='elec', problem_type = 'regression', eval_metric='mean_squared_error',
                            path = save_path)
ag_fit={"ag.max_memory_usage_ratio":1.5}
predictor.fit(train_data, hyperparameters=hyperparameters, time_limit=3600, presets='medium_quality',ag_args_fit=ag_fit)
print("==================learning_complete========================")

==================Tabular_complete========================


ValueError: Unknown `.fit` keyword argument specified: 'ag_ars_fit'
Valid kwargs: ['ag_args', 'ag_args_ensemble', 'ag_args_fit', 'auto_stack', 'calibrate', 'ds_args', 'excluded_model_types', 'feature_generator', 'feature_prune_kwargs', 'holdout_frac', 'hyperparameter_tune_kwargs', 'included_model_types', 'keep_only_best', 'name_suffix', 'num_bag_folds', 'num_bag_sets', 'num_stack_levels', 'pseudo_data', 'refit_full', 'save_bag_folds', 'save_space', 'set_best_to_refit_full', 'unlabeled_data', 'use_bag_holdout', 'verbosity']

In [51]:
predictor.leaderboard(train_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order


In [52]:
test

,electric_test.num,electric_test.tm,electric_test.hh24,electric_test.stn,electric_test.nph_ta,electric_test.nph_hm,electric_test.nph_ws_10m,electric_test.nph_rn_60m,electric_test.nph_ta_chi,electric_test.weekday,electric_test.week_name
0,2385,2023-01-01 01:00:00,1,303,7.8,61.5,6.7,0.0,4.2,6,1
1,2385,2023-01-01 02:00:00,2,303,7.9,60.6,7.6,0.0,4.0,6,1
2,2385,2023-01-01 03:00:00,3,303,8.2,61.9,8.7,0.0,4.1,6,1
3,2385,2023-01-01 04:00:00,4,303,8.4,60.9,9.2,0.0,4.3,6,1
4,2385,2023-01-01 05:00:00,5,303,8.5,60.9,9.5,0.0,4.3,6,1
...,...,...,...,...,...,...,...,...,...,...,...
2838234,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1
2838235,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1
2838236,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1
2838237,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1


In [53]:
for idx, col in enumerate(test.columns):
    print(idx, col)
    test.rename(columns={col: col.replace("electric_test.", "")}, inplace=True)

0 electric_test.num
1 electric_test.tm
2 electric_test.hh24
3 electric_test.stn
4 electric_test.nph_ta
5 electric_test.nph_hm
6 electric_test.nph_ws_10m
7 electric_test.nph_rn_60m
8 electric_test.nph_ta_chi
9 electric_test.weekday
10 electric_test.week_name


In [54]:
test['hour'] = pd.to_datetime(test['tm']).dt.hour
test['month'] = pd.to_datetime(test['tm']).dt.month
test['day'] = pd.to_datetime(test['tm']).dt.day
test['n']=0

In [55]:
# take log on test 
for col in ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']:
    test[col+'_log'] = np.log(test[col].abs()+1)
    test[col+'_log'] = np.where(test[col+'_log']<0, -test[col+'_log'], test[col+'_log'])
    test[col+'_log'] = np.where(test[col+'_log']==0, 0, test[col+'_log'])

In [56]:
test.drop('tm', axis=1, inplace=True)

In [57]:
train

,num,hh24,n,stn,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elec,hour,month,day,nph_ta_log,nph_hm_log,nph_ws_10m_log,nph_rn_60m_log,nph_ta_chi_log
0,4821,1,11,884,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,1,1,1,1.163151,4.154185,1.029619,0.0,0.693147
1,4821,2,11,884,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2,1,1,1.193922,4.160444,1.131402,0.0,0.470004
2,4821,3,11,884,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,3,1,1,1.163151,4.149464,1.252763,0.0,0.832909
3,4821,4,11,884,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,4,1,1,0.993252,4.166665,0.993252,0.0,0.182322
4,4821,5,11,884,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,5,1,1,0.993252,4.158883,0.955511,0.0,0.587787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,20,23,671,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,20,12,31,1.308333,3.856510,1.410987,0.0,0.336472
7593351,20947,21,23,671,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,21,12,31,1.280934,3.867026,1.410987,0.0,0.405465
7593352,20947,22,23,671,2.4,47.4,2.1,0.0,0.2,5,1,129.33,22,12,31,1.223775,3.879500,1.131402,0.0,0.182322
7593353,20947,23,23,671,2.5,47.0,2.1,0.0,0.3,5,1,122.57,23,12,31,1.252763,3.871201,1.131402,0.0,0.262364


In [58]:
test_data = TabularDataset(test)
y_pred = predictor.predict(test_data)

AssertionError: Trainer has no fit models that can infer.

# 느낀점
1. 데이터 사이즈가 커서 automl 쓰기 힘들수도있을 것 같다
2. EDA를 해보진 않았는데, 데이터에 대한 이해가 좀더 필요할 것 같다
3. 대부분 수치형데이터라 상관관계 등을 고려한 분석이 이어지면 좋을 것 같다. 


In [ ]:
y_pred.to_csv('electric_result_log.csv')

0 electric_test.num
1 electric_test.tm
2 electric_test.hh24
3 electric_test.stn
4 electric_test.nph_ta
5 electric_test.nph_hm
6 electric_test.nph_ws_10m
7 electric_test.nph_rn_60m
8 electric_test.nph_ta_chi
9 electric_test.weekday
10 electric_test.week_name


In [ ]:
test.columns

Index(['num', 'tm', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m',
       'nph_rn_60m', 'nph_ta_chi', 'weekday', 'week_name'],
      dtype='object')